In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

api_key = os.getenv('GROQ_API_KEY')
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [3]:
from langchain_groq import ChatGroq

In [4]:
llm = ChatGroq(model="llama-3.1-70b-versatile",
               api_key=api_key,
               temperature=0.6)

In [5]:
from langchain.document_loaders import CSVLoader

In [6]:
loader = CSVLoader('codebasics_faqs.csv',source_column='prompt')
data = loader.load()

In [7]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [ ]:
embeddings = HuggingFaceEmbeddings()

In [9]:
from langchain.vectorstores import Chroma

In [10]:
vectordb = Chroma.from_documents(
    documents=data,
    embedding=embeddings
)

In [11]:
retriever = vectordb.as_retriever()

In [ ]:
retriever.invoke('how about job placement support')

In [13]:
from langchain.prompts import PromptTemplate

In [14]:
prompt_template = PromptTemplate.from_template(""" 
                    Given the following context and a question, generate an answer based on this content.
                    In the answer try to provide as much text as possible from "response" section in the source document
                    If the answer is not found in the context, kindly state "I dont know". Do not make up an answer.
                    

                    CONTEXT: {context}

                    QUESTION: {question}           
                                    
                    """)

In [15]:
from langchain.chains import RetrievalQA

In [16]:
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=retriever,
    input_key="query",
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt_template}
)

In [ ]:
chain('what does this course teach')

In [ ]:
chain.invoke('what does this course teach')